### Langserve  Introduction
LangServe helps developers deploy LangChain runnables and chains as a REST API.
<br>

This library is integrated with FastAPI and uses pydantic for data validation<br>


https://github.com/langchain-ai/langserve/tree/mai<br>
packages.txt review for necessary libraries
<br>
### Install

pip install "langserve[all]"

### FastAPI
https://fastapi.tiangolo.com/

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.8+ based on standard Python type hints.

### Pydantic
https://docs.pydantic.dev/latest/

Pydantic is the most widely used data validation library for Python.

### Anthropic
https://www.anthropic.com/api



In [ ]:
import uuid
from langserve import RemoteRunnable

conversation_id = str(uuid.uuid4())
chat = RemoteRunnable("http://localhost:8000/", cookies={"user_id": "olonok"})

In [ ]:
conversation_id

In [ ]:
chat.invoke({"human_input": "my name is olonok. what is your name?"}, {'configurable': { 'conversation_id': conversation_id } })


In [ ]:
chat.invoke({"human_input": "what was my name?"}, {'configurable': { 'conversation_id': conversation_id } })

In [ ]:
# USe another userId

In [ ]:
chat = RemoteRunnable("http://localhost:8000/", cookies={"user_id": "pajaro"})

In [ ]:

chat.invoke({"human_input": "Como me llamo?"}, {'configurable': { 'conversation_id': conversation_id }})

In [ ]:
for chunk in chat.stream({'human_input': "Can you tell me the days of the week in German?"},  {'configurable': { 'conversation_id': conversation_id } }):
    print()
    print(chunk.content, end='', flush=True)

In [ ]:
conversation_id

In [ ]:
!tree chat_histories/

In [ ]:
import json

In [ ]:
with open("chat_histories/olonok/4ed45ab3-a34b-40cc-bebe-7226389668c9.json", "r") as f:
    c = json.load(f)

In [ ]:
c

In [ ]:
with open("chat_histories/pajaro/4ed45ab3-a34b-40cc-bebe-7226389668c9.json", "r") as f:
    d = json.load(f)

In [ ]:
d

In [ ]:
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langserve import RemoteRunnable

openai = RemoteRunnable("http://localhost:8000/openai/")
anthropic = RemoteRunnable("http://localhost:8000/anthropic/")
joke_chain = RemoteRunnable("http://localhost:8000/joke/")

#joke_chain.invoke({"topic": "parrots"})

# or async
await joke_chain.ainvoke({"topic": "parrots"})



In [ ]:
prompt = [
    SystemMessage(content='Act like either a cat or a parrot.'),
    HumanMessage(content='Hello!')
]

# Supports astream
async for msg in anthropic.astream(prompt):
    print(msg, end="", flush=True)



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [("human", "Tell me a long story about {topic}")]
)

# Can define custom chains
chain = prompt | RunnableMap({
    "openai": openai,
     "anthropic": anthropic,
})

chain.batch([{"topic": "parrots"}, {"topic": "cats"}])

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/joke/invoke",
    json={'input': {'topic': 'taxi-drivers'}}
)
response.json()